In [115]:
import pandas as pd
import pickle
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [116]:
# Read the CSV file into a DataFrame
df = pd.read_csv('condition_filled.csv')

In [117]:
data = df[["ID mosta", 'n_2012', 'n_2013', 'n_2014',
       'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019', 'n_2020', 'n_2021',
       'n_2022', 'n_2023', 'n_length', 'n_width', 'n_region', 'n_class',
       'n_years', 'n_material', 'n_type', 'b_change' ]]

data_lt = data[data["b_change"] == -1]
data_eq = data[data["b_change"] == 0]
data_gt = data[data["b_change"] == 1]

# Take 1000 random rows from data_eq
random_indices = np.random.choice(data_eq.index, size=1000, replace=False)
random_data_eq = data_eq.loc[random_indices]

# Concatenate random_data_eq with data_gt
new_df = pd.concat([random_data_eq, data_gt], ignore_index=True)

predict = "n_2023"
droplist = ["ID mosta", 'b_change', predict]

X = new_df.drop(droplist, axis=1)
y = new_df[predict]

n_layers = X.shape[1]

In [118]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [119]:
from sklearn.preprocessing import MinMaxScaler

# Create a Min-Max scaler
scaler = MinMaxScaler()

# Fit the scaler to your training data and transform both training and test data
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

In [144]:
for i in range(800, 1201, 50):
    data = df[["ID mosta", 'n_2012', 'n_2013', 'n_2014',
        'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019', 'n_2020', 'n_2021',
        'n_2022', 'n_2023', 'n_length', 'n_width', 'n_region', 'n_class',
        'n_years', 'n_material', 'n_type', 'b_change' ]]

    data_lt = data[data["b_change"] == -1]
    data_eq = data[data["b_change"] == 0]
    data_gt = data[data["b_change"] == 1]

    # Take 1000 random rows from data_eq
    random_indices = np.random.choice(data_eq.index, size=i, replace=False)
    random_data_eq = data_eq.loc[random_indices]

    # Concatenate random_data_eq with data_gt
    new_df = pd.concat([random_data_eq, data_gt], ignore_index=True)

    predict = "n_2023"
    droplist = ["ID mosta", 'b_change', predict]

    X = new_df.drop(droplist, axis=1)
    y = new_df[predict]

    n_layers = X.shape[1]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    from sklearn.preprocessing import MinMaxScaler

    # Create a Min-Max scaler
    scaler = MinMaxScaler()

    # Fit the scaler to your training data and transform both training and test data
    X_train_normalized = scaler.fit_transform(X_train)
    X_test_normalized = scaler.transform(X_test)

    #### Delete up until here ################

    # Define the model architecture using the number of normalized features
    model = keras.Sequential([
        keras.layers.Input(shape=(n_layers,)),  # Number of input features
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(1)
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model on the normalized training data
    model.fit(X_train_normalized, y_train, epochs=250, batch_size=8, validation_split=0.1, verbose=False)

    # Evaluate the model on the normalized test data
    loss = model.evaluate(X_test_normalized, y_test)
    # print(f"Mean Squared Error on Test Data: {loss}")

    #### Delete from here ################

    X = data_gt.drop(droplist, axis=1)
    y = data_gt[predict]
    X_normalized = scaler.transform(X)

    # Make predictions on the normalized test data
    predictions = model.predict(X_normalized)

    # Round the predicted values to zero decimal places and convert to integers
    rounded_predictions = np.round(predictions).astype(int)

    # Create a DataFrame to compare the actual 'n_2023' values with the model's predictions
    comparison_gt = pd.DataFrame({'Actual': y, 'Predicted': rounded_predictions.flatten()})


    # Add a new column "check" to the comparison DataFrame
    comparison_gt['check'] = comparison_gt['Actual'] == comparison_gt['Predicted']

    # Display the comparison DataFrame
    print(comparison_gt["check"].value_counts())
    print(f"Percentage of correct predictions: {round(comparison_gt['check'].mean()*100, 2)}%")




    X = data_eq.drop(droplist, axis=1)
    y = data_eq[predict]
    X_normalized = scaler.transform(X)

    # Make predictions on the normalized test data
    predictions = model.predict(X_normalized)

    # Round the predicted values to zero decimal places and convert to integers
    rounded_predictions = np.round(predictions).astype(int)

    # Create a DataFrame to compare the actual 'n_2023' values with the model's predictions
    comparison_df = pd.DataFrame({'Actual': y, 'Predicted': rounded_predictions.flatten()})


    # Add a new column "check" to the comparison DataFrame
    comparison_df['check'] = comparison_df['Actual'] == comparison_df['Predicted']

    # Display the comparison DataFrame
    print(comparison_df["check"].value_counts())
    print(f"Percentage of correct predictions: {round(comparison_df['check'].mean()*100, 2)}%")

    # Write results to text file
    with open('results.txt', 'a') as f:
        f.write(f"Iteration {i}:\n")
        f.write(f"Percentage of correct predictions for data_eq: {round(comparison_df['check'].mean()*100, 2)}%\n") 
        f.write(f"Percentage of correct predictions for data_gt: {round(comparison_gt['check'].mean()*100, 2)}%\n") 
        f.write("\n")  # Add a newline to separate results for different iterations

32/32 [==============================] - 0s 616us/step
check
True     578
False    415
Name: count, dtype: int64
Percentage of correct predictions: 58.21%
201/201 [==============================] - 0s 387us/step
check
True     4494
False    1911
Name: count, dtype: int64
Percentage of correct predictions: 70.16%
32/32 [==============================] - 0s 444us/step
check
True     637
False    356
Name: count, dtype: int64
Percentage of correct predictions: 64.15%
201/201 [==============================] - 0s 351us/step
check
True     4345
False    2060
Name: count, dtype: int64
Percentage of correct predictions: 67.84%
32/32 [==============================] - 0s 512us/step
check
True     653
False    340
Name: count, dtype: int64
Percentage of correct predictions: 65.76%
201/201 [==============================] - 0s 404us/step
check
True     3890
False    2515
Name: count, dtype: int64
Percentage of correct predictions: 60.73%
32/32 [==============================] - 0s 422us/step
che

In [134]:
# data = df[["ID mosta", 'n_2012', 'n_2013', 'n_2014',
#        'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019', 'n_2020', 'n_2021',
#        'n_2022', 'n_2023', 'n_length', 'n_width', 'n_region', 'n_class',
#        'n_years', 'n_material', 'n_type', 'b_change' ]]

# data = data[data["b_change"] == 1]

# predict = "n_2023"
# droplist = ["ID mosta", 'b_change', predict]

X = data_gt.drop(droplist, axis=1)
y = data_gt[predict]
X_normalized = scaler.transform(X)

# Make predictions on the normalized test data
predictions = model.predict(X_normalized)

# Round the predicted values to zero decimal places and convert to integers
rounded_predictions = np.round(predictions).astype(int)

# Create a DataFrame to compare the actual 'n_2023' values with the model's predictions
comparison_df = pd.DataFrame({'Actual': y, 'Predicted': rounded_predictions.flatten()})


# Add a new column "check" to the comparison DataFrame
comparison_df['check'] = comparison_df['Actual'] == comparison_df['Predicted']

# Display the comparison DataFrame
print(comparison_df["check"].value_counts())
print(f"Percentage of correct predictions: {round(comparison_df['check'].mean()*100, 2)}%")

X = data_eq.drop(droplist, axis=1)
y = data_eq[predict]
X_normalized = scaler.transform(X)

# Make predictions on the normalized test data
predictions = model.predict(X_normalized)

# Round the predicted values to zero decimal places and convert to integers
rounded_predictions = np.round(predictions).astype(int)

# Create a DataFrame to compare the actual 'n_2023' values with the model's predictions
comparison_df = pd.DataFrame({'Actual': y, 'Predicted': rounded_predictions.flatten()})


# Add a new column "check" to the comparison DataFrame
comparison_df['check'] = comparison_df['Actual'] == comparison_df['Predicted']

# Display the comparison DataFrame
print(comparison_df["check"].value_counts())
print(f"Percentage of correct predictions: {round(comparison_df['check'].mean()*100, 2)}%")
       

32/32 [==============================] - 0s 352us/step
check
True     646
False    347
Name: count, dtype: int64
Percentage of correct predictions: 65.06%


In [138]:
for i in range(800, 1201, 50):
    print(f"i = {i}")

i = 800
i = 850
i = 900
i = 950
i = 1000
i = 1050
i = 1100
i = 1150
i = 1200


In [411]:
# with open("ML_sts.pickle", "wb") as f:
#     pickle.dump(best_model, f)